In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import random

In [2]:
class GridWorld:
    """Grid world environment for Q-Learning"""

    def __init__(self, size=5):
        self.size = size
        self.start = (0, 0)
        self.goal = (size-1, size-1)
        self.obstacles = [(1, 1), (2, 2), (3, 1)]
        self.state = self.start

    def reset(self):
        """Reset environment to starting state"""
        self.state = self.start
        return self.state

In [3]:
def step(self, action):
    """Execute action and return next state, reward, done"""
    x, y = self.state

    # Actions: 0=up, 1=right, 2=down, 3=left
    if action == 0:  # up
        x = max(0, x - 1)
    elif action == 1:  # right
        y = min(self.size - 1, y + 1)
    elif action == 2:  # down
        x = min(self.size - 1, x + 1)
    elif action == 3:  # left
        y = max(0, y - 1)

    next_state = (x, y)

    # Check if hit obstacle
    if next_state in self.obstacles:
        next_state = self.state  # Stay in place
        reward = -10
    elif next_state == self.goal:
        reward = 100
    else:
        reward = -1  # Small penalty for each step

    self.state = next_state
    done = (next_state == self.goal)

    return next_state, reward, done

# Add this method to GridWorld class
GridWorld.step = step